# Data-Driven Tuning the Control Gains using Optimization

Use an optimization algorithm (like CMA-ES, Bayesian optimization, or Genetic Algorithms) to optimize the 44 entries in K

In [1]:
import numpy as np
import pandas as pd
from scipy.signal import savgol_filter
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import RectBivariateSpline


params = {
    'ms': 1500.0,       # Sprung mass (kg)
    'I_theta': 2500.0,  # Pitch moment of inertia (kg*m^2)
    'I_phi': 500.0,     # Roll moment of inertia (kg*m^2)
    'mu': [50.0, 50.0, 50.0, 50.0], # Unsprung masses for each wheel (kg)
    'ks': 25000.0,      # Suspension stiffness (N/m)
    'cs': 2000.0,       # Suspension damping (N*s/m)
    'kt': 200000.0,     # Tire stiffness (N/m)
    'ct': 150.0,        # Tire damping (N*s/m)
    'lf': 1.35,         # Distance from CG to front axle (m)
    'lr': 1.35,         # Distance from CG to rear axle (m)
    'tf': 0.75,         # Half-track width of front axle (m)
    'tr': 0.75,         # Half-track width of rear axle (m)
    'h_cg': 0.55        # height of the CG above the ground (m) for a mid-size passenger vehicle
}

def vehicle_dynamics(t, x, params, u=None, ax=0, vx=0, steering_angle=0, z_r=None, z_r_dot=None):
    """
    Calculates the time derivative of the vehicle's state vector.

    Args:
        t (float): Current time.
        x (np.ndarray): The 14-dimensional state vector.
                        x = [zs, theta, phi, zu1, zu2, zu3, zu4,
                             zs_dot, theta_dot, phi_dot, zu1_dot, zu2_dot, zu3_dot, zu4_dot]
        params (dict): A dictionary containing vehicle parameters:
                       'ms': sprung mass
                       'I_theta': pitch moment of inertia
                       'I_phi': roll moment of inertia
                       'mu': unsprung masses (list or array of 4 elements)
                       'ks': suspension stiffness
                       'cs': suspension damping
                       'kt': tire stiffness
                       'ct': tire damping
                       'lf': distance from CG to front axle
                       'lr': distance from CG to rear axle
                       'tf': half-track width of front axle
                       'tr': half-track width of rear axle
                       'h_cg': height of the CG above the ground 
        u (np.ndarray, optional): 4-element array of control inputs u1, u2, u3, u4. Defaults to [0,0,0,0].
        ax: longitudinal acceleration (e.g., from accelerating and braking)
        vx: vehicle forward speed (in m/s)
        steering_angle: steering angle (in rad)
        z_r (np.ndarray, optional): 4-element array of road displacements z_r1, z_r2, z_r3, z_r4. Defaults to [0,0,0,0].
        z_r_dot (np.ndarray, optional): 4-element array of road velocities z_r_dot1, etc. Defaults to [0,0,0,0].

    Returns:
        np.ndarray: The 14-dimensional time derivative of the state vector (x_dot).
    """

    # Unpack state variables
    zs, theta, phi, zu1, zu2, zu3, zu4, \
        zs_dot, theta_dot, phi_dot, zu1_dot, zu2_dot, zu3_dot, zu4_dot = x

    # Unpack parameters
    ms = params['ms']
    I_theta = params['I_theta']
    I_phi = params['I_phi']
    mu = params['mu']  # A list/array of [mu1, mu2, mu3, mu4]
    ks = params['ks']
    cs = params['cs']
    kt = params['kt']
    ct = params['ct']
    lf = params['lf']
    lr = params['lr']
    tf = params['tf']
    tr = params['tr']
    h_cg = params['h_cg']

    # Default control inputs and road inputs if not provided
    if u is None:
        u = np.zeros(4)
    if z_r is None:
        z_r = np.zeros(4)
    if z_r_dot is None:
        z_r_dot = np.zeros(4)

    # State derivatives - first part are simply the current velocities
    x_dot = np.zeros(14)
    x_dot[0] = zs_dot
    x_dot[1] = theta_dot
    x_dot[2] = phi_dot
    x_dot[3] = zu1_dot
    x_dot[4] = zu2_dot
    x_dot[5] = zu3_dot
    x_dot[6] = zu4_dot

    # Calculate Delta_i and Delta_dot_i
    Delta = np.array([
        -lf * theta + tf * phi,
        -lf * theta - tf * phi,
        lr * theta + tr * phi,
        lr * theta - tr * phi
    ])

    Delta_dot = np.array([
        -lf * theta_dot + tf * phi_dot,
        -lf * theta_dot - tf * phi_dot,
        lr * theta_dot + tr * phi_dot,
        lr * theta_dot - tr * phi_dot
    ])

    # Combine unsprung mass vertical positions and velocities for easier indexing
    zu = np.array([zu1, zu2, zu3, zu4])
    zu_dot = np.array([zu1_dot, zu2_dot, zu3_dot, zu4_dot])

    # Calculate F_Si (Suspension forces)
    F_Si = np.zeros(4)
    for i in range(4):
        F_Si[i] = ks * (zu[i] - zs - Delta[i]) + cs * (zu_dot[i] - zs_dot - Delta_dot[i])

    # Calculate F_Ti (Tire forces)
    F_Ti = np.zeros(4)
    for i in range(4):
        F_Ti[i] = kt * (z_r[i] - zu[i]) + ct * (z_r_dot[i] - zu_dot[i])    

    # Calculate accelerations (the second part of x_dot)

    # Sprung Mass Vertical Motion (ddot_zs)
    sum_F_S_plus_u = np.sum(F_Si + u)
    x_dot[7] = sum_F_S_plus_u / ms  # ddot_zs

    # Pitch Motion (ddot_theta)
    x_i = np.array([-lf, -lf, lr, lr])
    sum_pitch_terms = np.sum((F_Si + u) * x_i) + ms * h_cg * ax
    x_dot[8] = sum_pitch_terms / I_theta  # ddot_theta

    # Roll Motion (ddot_phi)
    y_i = np.array([tf, -tf, tr, -tr])
    ay= vx**2*np.tan(steering_angle)/(lf+lr)
    sum_roll_terms = np.sum((F_Si + u) * y_i) + ms * h_cg * ay
    x_dot[9] = sum_roll_terms / I_phi  # ddot_phi

    # Unsprung Mass (Wheel) Motion (ddot_zui)
    for i in range(4):
        x_dot[10 + i] = (F_Ti[i] - F_Si[i] - u[i]) / mu[i]  # ddot_zu1, ddot_zu2, ddot_zu3, ddot_zu4

    return x_dot



In [2]:
t_start = 0.0
t_end = 10.0
dt = 0.01

num_points = int(t_end/dt)+1
t = np.linspace(t_start, t_end, num_points)
df_suspension_data_aggr = pd.read_csv('driving_wheel_data_aggr.csv')

# Extracting time values
t = df_suspension_data_aggr.iloc[:, 0].values  # Time column

# Dictionary to store the interpolation functions
interp_funcs = {}

# Loop through columns 2 to 12 (index 1 to 11)
for i in range(1, df_suspension_data_aggr.shape[1]):
    col_name = df_suspension_data_aggr.columns[i]
    data = df_suspension_data_aggr.iloc[:, i].values
    interp_funcs[col_name] = interp1d(t, data, kind='cubic', fill_value="extrapolate")

def dynamic_system_wrapper(x, t, params, u_fixed):
    # This wrapper allows odeint to pass `t` as the second argument
    # and includes time-varying road inputs.
    z_r_t = np.zeros(4)
    z_r_dot_t = np.zeros(4)

    # # Apply road input to specific wheels (e.g., wheel 1)
    # z_r_t[0] = z_r_interp(t)
    # z_r_dot_t[0] = z_r_dot_interp(t)
    # ax = float(ax_interp(t))
    # vx = float(vx_interp(t))

    z_r_t[0] = interp_funcs['front_left_z'](t)
    z_r_dot_t[0] = interp_funcs['front_left_dz_dt'](t)
    z_r_t[1] = interp_funcs['front_right_z'](t)
    z_r_dot_t[1] = interp_funcs['front_right_dz_dt'](t)
    z_r_t[2] = interp_funcs['rear_left_z'](t)
    z_r_dot_t[2] = interp_funcs['rear_left_dz_dt'](t)
    z_r_t[3] = interp_funcs['rear_right_z'](t)
    z_r_dot_t[3] = interp_funcs['rear_right_dz_dt'](t)

    ax = float(interp_funcs['ax'](t))
    vx = float(interp_funcs['vx'](t))
    steering = float(interp_funcs['steering'](t))

    return vehicle_dynamics(t, x, params, u=u_fixed, ax=ax, vx=vx, steering_angle=steering, z_r=z_r_t, z_r_dot=z_r_dot_t)

def dynamic_system_control_wrapper(x, t, params, K):
    # This wrapper allows odeint to pass `t` as the second argument
    # and includes time-varying road inputs.
    z_r_t = np.zeros(4)
    z_r_dot_t = np.zeros(4)

    # # Apply road input to specific wheels (e.g., wheel 1)
    # z_r_t[0] = z_r_interp(t)
    # z_r_dot_t[0] = z_r_dot_interp(t)
    # ax = float(ax_interp(t))
    # vx = float(vx_interp(t))

    z_r_t[0] = interp_funcs['front_left_z'](t)
    z_r_dot_t[0] = interp_funcs['front_left_dz_dt'](t)
    z_r_t[1] = interp_funcs['front_right_z'](t)
    z_r_dot_t[1] = interp_funcs['front_right_dz_dt'](t)
    z_r_t[2] = interp_funcs['rear_left_z'](t)
    z_r_dot_t[2] = interp_funcs['rear_left_dz_dt'](t)
    z_r_t[3] = interp_funcs['rear_right_z'](t)
    z_r_dot_t[3] = interp_funcs['rear_right_dz_dt'](t)

    ax = float(interp_funcs['ax'](t))
    vx = float(interp_funcs['vx'](t))
    steering = float(interp_funcs['steering'](t))

    # Assuming x = [z_s, theta, phi, z_u1, z_u2, z_u3, z_u4, ..., z_s_dot, theta_dot, ...]
    z_s = x[0]
    z_u = x[3:7]  # wheel vertical displacements
    z_s_dot = x[7]
    theta_dot = x[8]
    phi_dot = x[9]

    # Suspension deflections and velocities
    suspension_deflection = z_s - z_u  # 4 values

    # Construct feedback state y
    y = np.concatenate(([z_s_dot],        # make scalar into 1D array
        [theta_dot],      # scalar → [scalar]
        [phi_dot]
        , z_u, suspension_deflection))  # y.shape = (11,)

    # --- Proportional control law: u = -K * y ---
    u_ctrl = -K @ y  # shape of K should be (4, 8) if u has 4 control inputs (1 per suspension)
    u_ctrl_clip = np.clip(u_ctrl, -3000, 3000)  # limit to ±3000 N
    # print(u_ctrl_clip)
    
    # Return dynamics
    return vehicle_dynamics(
        t, x, params,
        u=u_ctrl_clip,
        ax=ax,
        vx=vx,
        steering_angle=steering,
        z_r=z_r_t,
        z_r_dot=z_r_dot_t
    )

In [3]:
from scipy.integrate import odeint

def compute_acceleration(vel, dt):
    return np.gradient(vel, dt)


def callback(K_flat):
    print("Current K (flattened):", K_flat[:5], "...")  # Show first few values

def objective(K_var):
    
    # K = K_flat.reshape((4, 11))
    K = np.array([
        # z_s_dot, theta_dot, phi_dot,   z_u1, z_u2, z_u3, z_u4,   defl1,  defl2,  defl3,  defl4
        [  K_var[0],   K_var[1],   K_var[2],   K_var[3],    0.0,    0.0,    0.0,   K_var[4],     0.0,     0.0,     0.0 ],  # u1
        [  K_var[0],  -K_var[1],   K_var[2],      0.0, K_var[3],    0.0,    0.0,       0.0, K_var[4],     0.0,     0.0 ],  # u2
        [  K_var[0],   K_var[1],  -K_var[2],      0.0,    0.0, K_var[3],    0.0,       0.0,     0.0, K_var[4],     0.0 ],  # u3
        [  K_var[0],  -K_var[1],  -K_var[2],      0.0,    0.0,    0.0, K_var[3],       0.0,     0.0,     0.0, K_var[4] ],  # u4
    ])
    x0 = np.zeros(14)
    
    # simulate your system with u = -K @ x
    solution_w_control = odeint(dynamic_system_control_wrapper, x0, t, args=(params, K))
    # return a scalar performance score (lower is better)
    zs = solution_w_control[:, 0]
    pitch_angle = solution_w_control[:, 1]
    roll_angle = solution_w_control[:, 2]
    zu1 = solution_w_control[:, 3]
    zu2 = solution_w_control[:, 4]
    zu3 = solution_w_control[:, 5]
    zu4 = solution_w_control[:, 6]
    dzs = solution_w_control[:, 7]
    pitch_vel = solution_w_control[:, 8]
    roll_vel = solution_w_control[:, 9]
    dzu1 = solution_w_control[:, 10]
    dzu2 = solution_w_control[:, 11]
    dzu3 = solution_w_control[:, 12]
    dzu4 = solution_w_control[:, 13]
    az = compute_acceleration(dzs, dt)
    pitch_acc = compute_acceleration(pitch_vel, dt)
    roll_acc = compute_acceleration(roll_vel, dt)

    # Assume r_pitch = 1.0 m (longitude distance from CG)
    # Assume r_pitch = 0.5 m (latitude distance from CG)
    a_head = az + 1.0 * pitch_acc + 0.5 * roll_acc

    comfort_index = np.sqrt((10.0*az)**2 + (1.0*pitch_acc)**2 + (0.5*roll_acc)**2)

    rms_ci = np.sqrt(np.mean(comfort_index**2))

    print(f"K_var:[{K_var[0]:.4f},{K_var[1]:.4f},{K_var[2]:.4f},{K_var[3]:.4f},{K_var[4]:.4f}], Cost: {rms_ci:.4f}")

    return rms_ci


In [ ]:
# K = np.array([
#     # z_s_dot, theta_dot, phi_dot,   z_u1, z_u2, z_u3, z_u4,   defl1,  defl2,  defl3,  defl4
#     [  2000.0,   5000.0,   5000.0,   1000.0,    0.0,    0.0,    0.0,   1000.0,     0.0,     0.0,     0.0 ],  # u1
#     [  2000.0,  -5000.0,   5000.0,      0.0, 1000.0,    0.0,    0.0,       0.0, 1000.0,     0.0,     0.0 ],  # u2
#     [  2000.0,   5000.0,  -5000.0,      0.0,    0.0, 1000.0,    0.0,       0.0,     0.0, 1000.0,     0.0 ],  # u3
#     [  2000.0,  -5000.0,  -5000.0,      0.0,    0.0,    0.0, 1000.0,       0.0,     0.0,     0.0, 1000.0 ],  # u4
# ])
# K_var = np.array([2000,5000,5000,1000,1000])
K_var = np.array([49250.9112,-61.1945,39.1599,29544.3617,-56150.7657])
cost_test = objective(K_var)

In [ ]:
cost_test2 = objective(K_var*0.5)


K_var:[1000.0000,2500.0000,2500.0000,500.0000,500.0000], Cost: 0.2678


In [ ]:
from scipy.optimize import minimize
K0 = K_var
res = minimize(objective, K0, method='Nelder-Mead', callback=callback, options={'disp': True})  # or use other methods
K_opt = res.x

In [ ]:
from bayes_opt import BayesianOptimization

# Wrap your objective function for BayesianOptimization
def objective_bo(k0, k1, k2, k3, k4):
    K_var = np.array([k0, k1, k2, k3, k4])
    cost = objective(K_var)
    return -cost  # BO maximizes, so we negate cost

# Define parameter bounds (can adjust based on expected ranges)
pbounds = {
    'k0': (-5000, 5000),
    'k1': (-3000, 3000),
    'k2': (-3000, 3000),
    'k3': (-10000, 10000),
    'k4': (-10000, 10000),
}

# Create optimizer
optimizer = BayesianOptimization(
    f=objective_bo,
    pbounds=pbounds,
    verbose=2,   # Shows iteration progress
    random_state=1,
)

# Run optimization
optimizer.maximize(
    init_points=20,   # random exploration steps
    n_iter=100,       # total optimization steps
)

# Best result
print("Best design found:")
print(optimizer.max)


|   iter    |  target   |    k0     |    k1     |    k2     |    k3     |    k4     |
-------------------------------------------------------------------------------------
K_var:[-829.7800,1321.9470,-2999.3138,-3953.3485,-7064.8822], Cost: 4.6045
| 1         | -4.604499 | -829.7799 | 1321.9469 | -2999.313 | -3953.348 | -7064.882 |
K_var:[-4076.6141,-1882.4387,-926.6356,-2064.6505,776.3347], Cost: 105.0681
| 2         | -105.0681 | -4076.614 | -1882.438 | -926.6356 | -2064.650 | 776.33468 |
K_var:[-808.0549,1111.3170,-1773.2865,7562.3487,-9452.2481], Cost: 0.4394
| 3         | -0.439418 | -808.0548 | 1111.3170 | -1773.286 | 7562.3487 | -9452.248 |
K_var:[1704.6751,-496.1712,352.1390,-7192.2612,-6037.9702], Cost: 0.3196
| 4         | -0.319600 | 1704.6751 | -496.1711 | 352.13897 | -7192.261 | -6037.970 |
K_var:[3007.4457,2809.5695,-1119.4549,3846.4523,7527.7830], Cost: 0.7357
| 5         | -0.735655 | 3007.4456 | 2809.5694 | -1119.454 | 3846.4523 | 7527.7830 |
K_var:[3946.0666,-2489.7347